In [8]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from datetime import datetime
from pathlib import Path
from tqdm import tqdm 
import json
from pprint import pprint
from bson import ObjectId

sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Other Read Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [9]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/5_optimized/omero_museum"

In [10]:
connector = MongoDBConnector("omero_museum", silent=True)
db = connector.db

### RD18: _Retrieve all the artworks of a given author_

In [106]:
author_id = db.artworks.find_one({"author_ids.0": {"$exists": True}})["author_ids"][0]  # type: ignore

out = list(db.artworks.find({"author_ids": author_id}))
jprint([a for a in out])  # type: ignore

 [
    
     {
         _id : Mona Lisa
         date : 2025-09-02 09:28:18
         type : relief
         description : “Il Partenone racchiude un’armonica sintesi di utilità, solidità e piacevolezza”, Vitruvio da “De Architectura”.
         is_original : False
         size : 230
         period : Ancient
         location_name : Forlimpopoli
         author_ids :
         [
             68b572f8845fcdbabc1c8905
             68b572f8845fcdbabc1c8908
         ]
         tecniques : [Mosaic]
         comments_star_1 :
         [
            
             {
                 _id : 68bc6aad24580947f612262b
                 rating : 1
                 date : 2025-08-24 21:51:17
                 message : Folla perché prendere animo insistere battere. Mattino utile oggi giardino.
Cui rapido staccare fratello creare. Attendere ponte formare. Interesse sereno soffrire parlare staccare energia oro.
             }
            
             {
                 _id : 68bc6aad24580947f61225f5
    

### RD19: _Retrieve all activity of a given workshop type_

In [31]:
workshop_title = db.activities.find_one({"workshop_title": {"$exists": True}})["workshop_title"]  # type: ignore

out = list(db.activities.find({"workshop_title": workshop_title}))
jprint([a for a in out])  # type: ignore


 [
    
     {
         _id : 68bc6aad24580947f6121f00
         room : Impressionismo
         duration : 315
         enrolled : 19
         start_date : 2025-08-30 18:44:38
         capacity : 85
         ticket_ids :
         [
             68bc6aad24580947f6121d9e
             68bc6aad24580947f6121ef4
             68bc6aad24580947f6121e67
             68bc6aad24580947f6121dff
             68bc6aad24580947f6121da2
             68bc6aad24580947f6121d2c
             68bc6aad24580947f6121ec8
             68bc6aad24580947f6121e0a
         ]
         workshop_title : Impronte
     }
    
     {
         _id : 68bc6aad24580947f6121f08
         room : Greco e Romano
         duration : 240
         enrolled : 16
         start_date : 2025-09-01 13:49:32
         capacity : 71
         workshop_title : Impronte
     }
    
     {
         _id : 68bc6aad24580947f6121f48
         room : Greco e Romano
         duration : 60
         enrolled : 17
         start_date : 2025-09-04 14:35:04
    

### RD20: _Read all the surveys filled out in a specific day_

In [105]:
date_start = datetime(2021, 4, 22)
date_end = datetime(2021, 4, 23)

out = db.visitors.find(
        {"surveys.date_of_compilation": {"$gte": date_start, "$lte": date_end}},
        {"surveys": 1},
    
).hint("surveys.date_of_compilation_1")
jprint([__ for _ in out for __ in _["surveys"]])

 [
    
     {
         accompanying_persons_visit : famiglia
         date_of_compilation : 2021-04-22 00:00:00
         reason_for_visit : approfondire le mie conoscenze
         number_of_visits : 0
         return : None
         type_of_visit : visita libera
         title_of_studies : Diploma
         evaluation_of_experience : None
         evaluation_of_facility : None
         evaluation_of_visit : None
     }
    
     {
         accompanying_persons_visit : partner/coniuge
         date_of_compilation : 2021-04-16 00:00:00
         reason_for_visit : incontrare persone con interessi simili ai miei
         number_of_visits : 0
         return : Assolutamente Si
         type_of_visit : visita libera
         title_of_studies : Diploma
         evaluation_of_experience : 5
         evaluation_of_facility : 5
         evaluation_of_visit : 5
     }
    
     {
         accompanying_persons_visit : scolaresca
         date_of_compilation : 2021-04-28 00:00:00
         reason_fo

### RD21: _Retrieve information about a given laboratory activity, including the total number of reservations_

In [62]:
activity_id = db.activities.find_one({"ticket_ids.0": {"$exists": True}})["_id"]  # type: ignore

out = list(db.activities.aggregate([
    {"$match": {"_id": activity_id}},
    {"$addFields": {"total_reservations": {"$size": {"$ifNull": ["$ticket_ids", []]}}}}
]))
jprint([a for a in out])  # type: ignore

 [
    
     {
         _id : 68bc6aad24580947f6121f00
         room : Impressionismo
         duration : 315
         enrolled : 19
         start_date : 2025-08-30 18:44:38
         capacity : 85
         ticket_ids :
         [
             68bc6aad24580947f6121d9e
             68bc6aad24580947f6121ef4
             68bc6aad24580947f6121e67
             68bc6aad24580947f6121dff
             68bc6aad24580947f6121da2
             68bc6aad24580947f6121d2c
             68bc6aad24580947f6121ec8
             68bc6aad24580947f6121e0a
         ]
         workshop_title : Impronte
         total_reservations : 8
     }
 ]


### RD22: _Retrieve all the information about a limited event_ 

In [34]:
event_id = db.limited_events.find_one({"_id": {"$exists": True}})["_id"]  # type: ignore
out = list(db.limited_events.find({"_id": event_id}))
jprint([a for a in out])  # type: ignore


 [
    
     {
         _id : Accessibility and Universal Design in Museums
         capacity : 32
         type : exhibition
         start_date : 2025-09-01 00:29:39
         end_date : 2025-08-15 15:32:47
         artist : 1
         description : Esso camera valere vestire grosso esercito. Strumento teatro intorno labbro principale passo vino.
Offendere conservare importanza soltanto. Passo ospedale però giudicare. Voglia comodo creare.
         room_name : Rinascimentale
         author_ids :
         [
             4
             3
             13
             0
         ]
         artwork_titles : [The Persistence of Memory, The Night Watch, The Birth of Venus, American Gothic, The Great Wave off Kanagawa, Campbell's Soup Cans, Nighthawks, Guernica]
         ticket_ids :
         [
             68bc6aad24580947f6121e2d
         ]
     }
 ]


### RD23: _Retrieve all the details of a visitor_ 

In [63]:
visitor_id = db.visitors.find_one({"_id": {"$exists": True}})["_id"]  # type: ignore

out = db.visitors.find_one({"_id": visitor_id})
jprint([out])  # type: ignore

 [
    
     {
         _id : 68bc6aad24580947f6121ca8
         is_customer : False
         birth_date : 2025-08-27 14:07:49
         impairment : blind
         gender : female
         land : Antigua e Barbuda
         surveys :
         [
            
             {
                 accompanying_persons_visit : gruppo organizzato
                 date_of_compilation : 2021-04-16 00:00:00
                 reason_for_visit : approfondire le mie conoscenze
                 number_of_visits : 0
                 return : Probabilmente No
                 type_of_visit : partecipazione ad una mostra/laboratorio
                 title_of_studies : Laurea/PostLaurea
                 evaluation_of_experience : None
                 evaluation_of_facility : None
                 evaluation_of_visit : None
             }
         ]
         donation_ids : [Mona Lisa]
         tickets : None
     }
 ]


### RD24: _Retrieve the visitor count for each land from most frequent to least frequent_ 

In [57]:
out = list(db.visitors.aggregate([
    {"$sortByCount": "$land"},
    {"$project": {"state": "$_id", "count": 1, "_id": 0}},
]))
jprint([r for r in out])  # type: ignore


 [
    
     {
         count : 3
         state : Myanmar
     }
    
     {
         count : 2
         state : Svalbard & Jan Mayen Islands
     }
    
     {
         count : 2
         state : Bulgaria
     }
    
     {
         count : 2
         state : Germania
     }
    
     {
         count : 2
         state : Togo
     }
    
     {
         count : 2
         state : Latvia
     }
    
     {
         count : 2
         state : Capo Verde
     }
    
     {
         count : 2
         state : American Samoa
     }
    
     {
         count : 2
         state : Regno Unito
     }
    
     {
         count : 2
         state : Guinea Equatoriale
     }
    
     {
         count : 2
         state : Niue
     }
    
     {
         count : 2
         state : Svezia
     }
    
     {
         count : 2
         state : Emirati Arabi Uniti
     }
    
     {
         count : 2
         state : Norvegia
     }
    
     {
         count : 1
         state : Iran
     }
  

### RD25: _Retrieve the visitor count for each impairment from most frequent to least frequent_ 

In [56]:
out = list(db.visitors.aggregate([
    {"$sortByCount": "$impairment"},
    {"$project": {"impairment": "$_id", "count": 1, "_id": 0}},
]))
jprint([r for r in out])  # type: ignore

 [
    
     {
         count : 45
         impairment : deaf
     }
    
     {
         count : 33
         impairment : blind
     }
    
     {
         count : 22
         impairment : None
     }
 ]


### RD26: _Count tickets issued for each year_ 

In [81]:
out = list(db.visitors.aggregate([
    {"$unwind": "$tickets"},
    {"$project": {"year": {"$year": "$tickets.date"}}},
    {"$sortByCount": "$year"},
    {"$project": {"year": "$_id", "count": 1, "_id": 0}}
], hint="tickets.date_1"))  

jprint([r for r in out])  # type: ignore

 [
    
     {
         count : 138
         year : 2025
     }
 ]


### RD27: _Calculate the distribution of visitor ages grouped into bands based on ticket activity in a specific year_ 

In [43]:
year = 2025
start_date = datetime(year, 1, 1)
end_date = datetime(year + 1, 1, 1)

out = list(db.visitors.aggregate([
    {
        "$match": {
            "birth_date": {"$ne": None},
            "tickets": {
                "$elemMatch": {
                    "date": {"$gte": start_date, "$lt": end_date}
                }
            }
        }
    },
    {
        "$addFields": {
            "valid_ticket": {
                "$first": {
                    "$filter": {
                        "input": "$tickets",
                        "as": "t",
                        "cond": {
                            "$and": [
                                {"$gte": ["$$t.date", start_date]},
                                {"$lt": ["$$t.date", end_date]}
                            ]
                        }
                    }
                }
            }
        }
    },
    {
        "$addFields": {
            "age": {
                "$dateDiff": {
                    "startDate": "$birth_date",
                    "endDate": "$valid_ticket.date",
                    "unit": "year"
                }
            }
        }
    },
    {
        "$addFields": {
            "age_group": {
                "$switch": {
                    "branches": [
                        {"case": {"$lte": ["$age", 18]}, "then": "0-18"},
                        {"case": {"$and": [{"$gt": ["$age", 18]}, {"$lte": ["$age", 30]}]}, "then": "19-30"},
                        {"case": {"$and": [{"$gt": ["$age", 30]}, {"$lte": ["$age", 45]}]}, "then": "31-45"},
                        {"case": {"$and": [{"$gt": ["$age", 45]}, {"$lte": ["$age", 65]}]}, "then": "46-65"},
                        {"case": {"$gt": ["$age", 65]}, "then": "65+"}
                    ],
                    "default": "unknown"
                }
            }
        }
    },
    {
        "$group": {
            "_id": "$age_group",
            "count": {"$sum": 1}
        }
    },
    {
        "$group": {
            "_id": None,
            "total": {"$sum": "$count"},
            "groups": {
                "$push": {
                    "age_group": "$_id",
                    "count": "$count"
                }
            }
        }
    },
    {"$unwind": "$groups"},
    {
        "$project": {
            "_id": 0,
            "age_group": "$groups.age_group",
            "count": "$groups.count",
            "percentage": {
                "$round": [
                    {"$multiply": [{"$divide": ["$groups.count", "$total"]}, 100]},
                    2
                ]
            }
        }
    },
    {"$sort": {"age_group": 1}}
], hint="tickets.date_1"))

jprint([r for r in out])  # type: ignore

 [
    
     {
         age_group : 0-18
         count : 85
         percentage : 100.0
     }
 ]


### RD28: _Count material usage in all artworks_ 

In [54]:
out = list(db.artworks.aggregate([
    {"$unwind": "$materials"},
    {"$sortByCount": "$materials"},
    {"$project": {"material": "$_id", "count": 1, "_id": 0}},
]))
jprint([r for r in out])  # type: ignore

 [
    
     {
         count : 7
         material : Bronze
     }
    
     {
         count : 6
         material : Textile
     }
    
     {
         count : 6
         material : Canvas
     }
    
     {
         count : 6
         material : Marble
     }
    
     {
         count : 5
         material : Wood
     }
    
     {
         count : 4
         material : Stone
     }
    
     {
         count : 4
         material : Paper
     }
    
     {
         count : 2
         material : Glass
     }
    
     {
         count : 2
         material : Steel
     }
    
     {
         count : 2
         material : Clay
     }
 ]


### RD29: _Count top 5 visit reasons_ 

In [76]:
out = list(db.visitors.aggregate([
    {"$unwind": "$surveys"},
    {"$sortByCount": "$surveys.reason_for_visit"},
    {"$limit": 5},
    {"$project": {"reason": "$_id", "count": 1, "_id": 0}},
]))
jprint([r for r in out])  # type: ignore



 [
    
     {
         count : 25
         reason : passare un momento personale piacevole
     }
    
     {
         count : 21
         reason : incontrare persone con interessi simili ai miei
     }
    
     {
         count : 21
         reason : vedere oggetti importanti
     }
    
     {
         count : 20
         reason : trascorrere tempo libero con amici/parenti
     }
    
     {
         count : 16
         reason : approfondire le mie conoscenze
     }
 ]


### RD30: _Count how many artworks are original vs. not_

In [67]:
pipeline = [
    {
        "$group": {
            "_id": "$is_original",
            "count": {"$sum": 1} # True = Original
            
        }
    },
    {
        "$sort": {"_id": -1} 
    },
    {
        "$project": {"Original": "$_id", "count": 1, "_id": 0}
    },
]


out = list(db.artworks.aggregate(pipeline))
jprint(out)

 [
    
     {
         count : 5
         Original : True
     }
    
     {
         count : 15
         Original : False
     }
 ]


### RD31: _Count how many visitors came back_

In [87]:
out = db.visitors.aggregate([
    {"$project": {
        "num_tickets": {"$size": {"$ifNull": ["$tickets", []]}}
    }},
    {"$match": {"num_tickets": {"$gt": 1}}},
    {"$count": "count"},
    {"$project": {"visitors returned": "$count", "_id": 0}}
])
jprint([r for r in out])  # type: ignore

 [
    
     {
         visitors returned : 32
     }
 ]


### RD32: _Count top 5 days with most ticket activity in a given year_

In [92]:
start = datetime(2025, 1, 1)
end = datetime(2026, 1, 1)

pipeline = [
    {"$match": {"tickets.date": {"$gte": start, "$lt": end}}},
    {"$unwind": "$tickets"},
    {"$project": {
        "day": {
            "$dateTrunc": {
                "date": "$tickets.date",
                "unit": "day"
            }
        }
    }},
    {"$group": {
        "_id": "$day",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}},
    {"$limit": 5},
    {"$project": {"day": "$_id", "count": 1, "_id": 0}}
]

out = list(db.visitors.aggregate(pipeline, hint="tickets.date_1"))

for doc in out:
    doc["day"] = doc["day"].strftime("%Y-%m-%d")
jprint(out)  # type: ignore

 [
    
     {
         count : 9
         day : 2025-08-27
     }
    
     {
         count : 8
         day : 2025-08-30
     }
    
     {
         count : 8
         day : 2025-08-21
     }
    
     {
         count : 8
         day : 2025-09-06
     }
    
     {
         count : 7
         day : 2025-08-22
     }
 ]


### RD33: _Average enrollment rate by workshop title_

In [104]:
out = list(db.activities.aggregate([
    {"$project": {
        "workshop_title": 1,
        "enrollment_rate": {
            "$cond": [
                {"$gt": ["$capacity", 0]},
                {"$divide": ["$enrolled", "$capacity"]},
                None
            ]
        }
    }},
    {"$group": {
        "_id": "$workshop_title",
        "avg_enrollment_rate": {"$avg": "$enrollment_rate"}
    }},
    {"$project": {
        "workshop_title": "$_id",
        "avg_enrollment_rate": 1,
        "_id": 0
    }}
]))
jprint([r for r in out])  # type: ignore


 [
    
     {
         avg_enrollment_rate : 0.19137866741553378
         workshop_title : Libri tattili
     }
    
     {
         avg_enrollment_rate : 0.17487593707105903
         workshop_title : Il Museo delle meraviglie
     }
    
     {
         avg_enrollment_rate : 0.21338957386978152
         workshop_title : Ri-tratto con tatto
     }
    
     {
         avg_enrollment_rate : 0.07272727272727272
         workshop_title : Ricordi da toccare
     }
    
     {
         avg_enrollment_rate : 0.3407862954487668
         workshop_title : Conosci Louis?
     }
    
     {
         avg_enrollment_rate : 0.10889704515163305
         workshop_title : Bestiario immaginario
     }
    
     {
         avg_enrollment_rate : 0.14492753623188406
         workshop_title : Le cose raccontano storie
     }
    
     {
         avg_enrollment_rate : 0.3053483642260732
         workshop_title : Di-segno
     }
    
     {
         avg_enrollment_rate : 0.3887147335423197
         workshop_

### RD34: _Count visitor education level breakdown from surveys in a given period_

In [73]:
start = datetime(2021, 1, 1)
end = datetime(2022, 1, 1)

out = list(db.visitors.aggregate([
    {"$match": {
        "surveys.date_of_compilation": {"$gte": start, "$lt": end}
    }},
    {"$project": {"titles": "$surveys.title_of_studies"}},
    {"$unwind": "$titles"},
    {"$group": {"_id": "$titles", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$project": {"title": "$_id", "count": 1, "_id": 0}},
], hint="surveys.date_of_compilation_1"))

jprint([r for r in out])  # type: ignore

 [
    
     {
         count : 36
         title : Laurea/PostLaurea
     }
    
     {
         count : 34
         title : Diploma
     }
    
     {
         count : 26
         title : None
     }
    
     {
         count : 18
         title : Elem.
     }
    
     {
         count : 7
         title : Lic. Media
     }
    
     {
         count : 2
         title : Nessuno
     }
 ]


### RD35: _Show workshop duration distribution_

In [74]:
out = list(db.workshops.aggregate([
    {"$sortByCount": "$duration"},
    {"$project": {
        "_id": 0,
        "duration": {
            "$dateToString": {
                "format": "%H:%M:%S",
                "date": "$_id"
            }
        },
        "count": 1
    }}
]))
jprint([r for r in out])  # type: ignore

 [
    
     {
         count : 8
         duration : 02:00:00
     }
    
     {
         count : 3
         duration : 01:30:00
     }
    
     {
         count : 1
         duration : 01:00:00
     }
 ]


### RD36: _Count the number of museums vs external suppliers_

In [65]:
pipeline = [
    {
        "$group": {
            "_id": "$is_museum",
            "count": {"$sum": 1}  # True = Museum
        }
    },
    {
        "$sort": {"_id": -1}
    },
    {   
        "$project": {"Museum": "$_id", "count": 1, "_id": 0}
    },
]

out = list(db.suppliers.aggregate(pipeline))
jprint(out)

 [
    
     {
         count : 46
         Museum : True
     }
    
     {
         count : 54
         Museum : False
     }
 ]
